In [ ]:
#join the owners duet session
import syft as sy
sy.VERBOSE = True
duet = sy.join_duet(loopback=True)

### Checkpoint 1. Return to Data Owner 1 

In [ ]:
#check the items currently in the duet store
duet.store.pandas

In [ ]:
#store the item in duet1_ptr
data1_ptr = duet.store[0]
print(data1_ptr)

In [ ]:
import torch
import time

In [ ]:
#set the input and output dimensions

#size of each input data
in_dim = 100
#size of each output data
out_dim = 100

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        #applies a linear transformation to the in/out dimensions
        self.linear = self.torch_ref.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x

In [ ]:
#Send one copy of the model to each data owner or client and train remotely
combined_model = SyNet(torch)

In [ ]:
def train(iterations, model, torch_ref, optim, data_ptr, target_ptr):
    
    losses = []
    
    for i in range(iterations):
        
        optim.zero_grad()
        
        output = model(data_ptr)

        #uses MSE (Mean Squared Error), to calculate loss
        loss = torch_ref.nn.functional.mse_loss(output, target_ptr)

        loss_item = loss.item()

        #send a request to get the actual value of the tensor from the Data Owner
        loss_value = loss_item.get(
                name="loss",
                reason="To evaluate training progress",
                request_block=True,
                timeout_secs=5
            )
        if i % 10 == 0:
            print("Epoch", i, "loss", loss_value)
        #append the loss value to the list
        losses.append(loss_value)
        loss.backward()
        optim.step()
    
    print ("Process took: ", time.time()-start, " seconds")
    return losses

In [ ]:
import torch as th
import numpy as np

### Begin Training on Data Owner 1

In [ ]:
local_model1 = SyNet(torch)

In [ ]:
print(local_model1.parameters())

In [ ]:
remote_model1 = local_model1.send(duet)

In [ ]:
remote_torch1 = duet.torch
params = remote_model1.parameters()
optim1 = remote_torch1.optim.Adam(params=params, lr=0.1)

In [ ]:
from skimage.transform import resize
from skimage.color import rgb2gray
#from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import imageio
import cv2 as cv

### Transform Target Data

In [ ]:
#loop through the directory and transform to tensor
path = "/Users/danielcastillo/Documents/senior-project/Data/CATS_DOGS/train/CAT"

#transform the images to a list of Tensors
transform(path)

In [ ]:
#change the list of tensors into a single tensor
target1_ptr = torch.stack(arr)
#change the tensor data type from Float64 to Float32
target1_ptr = target1_ptr.type(torch.FloatTensor)

In [ ]:
print(target1_ptr)
print(target1_ptr.shape)

### Start training on Data Owner 1 data

In [ ]:
#variable to keep track of training time
start = time.time()

iteration = 100
losses = train(iteration, remote_model1, remote_torch1, optim1, data1_ptr, target1_ptr)

# End of Calculation for DATA OWNER 1

In [ ]:
#join the owners duet session
import syft as sy
sy.VERBOSE = True
duet2 = sy.join_duet(loopback=True)

## checkpoint 2. return to Data Owner 2

In [ ]:
#check the items in the duet store
duet2.store.pandas

In [ ]:
#store the item in duet1_ptr
data2_ptr = duet2.store[0]
print(data2_ptr)

In [ ]:
local_model2 = SyNet(torch)

In [ ]:
print(local_model2.parameters())

In [ ]:
remote_model2 = local_model2.send(duet2)

In [ ]:
remote_torch2 = duet2.torch
params = remote_model2.parameters()
optim2 = remote_torch2.optim.Adam(params=params, lr=0.1)

### transform target data for Data Owner 2

In [ ]:
#loop through the directory and transform to tensor
path = "/Users/danielcastillo/Documents/senior-project/Data/CATS_DOGS/train/DOG"

#transform the images to a list of Tensors
transform(path)

In [ ]:
#change the list of tensors into a single tensor
target2_ptr = torch.stack(arr)
#change the tensor data type from Float64 to Float32
target2_ptr = target2_ptr.type(torch.FloatTensor)

In [ ]:
print(target2_ptr)
print(target2_ptr.shape)

### Start training on Data Owner 2 data

In [ ]:
#variable for tracking training time
start = time.time()

iteration = 100
losses = train(iteration, remote_model2, remote_torch2, optim2, data2_ptr, target2_ptr)

# End of Calculation for Data Owner 2

# Definition for transform()

In [ ]:
def transform(path):
    #declare a new list
    arr = []
    
    tqdm().pandas()
    i = 0


    for pic in tqdm(os.scandir(path)):
        #control the number of images to transform
        if i < 100:
            #check if file is of valid type 
            if (pic.path.endswith(".jpeg") or pic.path.endswith(".jpg")) and pic.is_file():
                #read the image
                picture = cv.imread(pic.path)
                if(type(picture) != None):
                    #resize the picture to 10 x 10
                    picture = resize(picture, (10,10), mode='constant', anti_aliasing=True)
                    #change the picture to gray scale
                    picture = rgb2gray(picture)
                    #transform the picture to a tensor
                    Tensor = torch.tensor(picture)
                    Tensor = Tensor.flatten()
                    #append the tensor to the list
                    arr.append(Tensor)
                else:
                    #this means that the picture was NoneType
                    print("not added to the list: ", i)
                    #skip iteration
                    continue
        i+=1 